# Sentiment Analysis

Here we demonstrate sentiment analysis on a group of student feedback comments. This involves classifying the comments as 'positive', 'negative', or 'neutral'. We'll also show an even finer-grained sorting from most positive to most negative, along with some nice color-coded output. 

Behind the scenes, if you are interested, we're using the model's logprobs (what were its first, second, third, etc. choices for the sentiment output token, and how confident was it in each of those possible choices) to get a sense of how confident the model was in its prediction and what else it considered. gpt-4 (post-RLHF, what we have access to) is somewhat well calibrated (it knows what it knows...kind of like metacognition for a model) and using the logprobs relies on this type of confidence.

## Imports and setup

In [1]:
import pandas as pd
import json
from typing import Any
from IPython.display import HTML
from functools import partial
from pprint import pprint
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from survey_analysis.sentiment_analysis import (
    SentimentAnalysis, 
    SentimentAnalysisResult, 
    classify_sentiment,
    sort_by_confidence,
)
from survey_analysis.models_common import CommentModel, LLMConfig
from survey_analysis.single_input_task import apply_task_with_logprobs
from survey_analysis.batch_runner import process_tasks

In [2]:
# this makes it more robust to run async tasks inside an already async environment (jupyter notebooks)
import nest_asyncio
nest_asyncio.apply()

Make sure to either set `OPENAI_API_KEY` as an environment variable or put it in a .env file and use the following cell to load the env var. The format in the .env file is:
```
OPENAI_API_KEY=yourKeyGoesHere
```

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
%load_ext autoreload
%autoreload 2

This is a convenience function to make seeing Pandas dataframe values easier, especially when there are long strings like the student comments we will be using.

In [5]:
def full_show(df):
    with pd.option_context('display.max_columns', None, 'display.max_rows', None, 'display.max_colwidth', None):
        display(df)

## Load the example data

In [6]:
data_path = Path('../data/example_data')

Let's load up some fake data. 

All of these comments are synthetic to avoid sharing any sensitive or PII information, but they should work great for illustration purposes. There are 100 rows, with just a few null/nan values here and there for realism. In most surveys I've seen, there are quite a number of null/None/blank etc values, and the functions are written to handle those.

In [7]:
example_survey = pd.read_csv(data_path / 'example_survey_data_synthetic.csv')
full_show(example_survey.head())

,best_parts,enhanced_learning,improve_course
0,I valued the practical clinical aspects related to immune-related disorders and their management.,The illustrative visuals and straightforward explanatory clips.,Consider reducing the duration of certain videos. A few appeared to be slightly prolonged.
1,The flexibility to learn at a self-determined speed,The opportunity to review the lecture content,"The pace of some lectures could be slowed down. At times, it's challenging to follow the lecturer's speech or decipher their handwriting."
2,The educational content was extremely enriching and stimulating! The section on oncology was the highlight.,the self-assessment activities.,Nothing specific comes to mind.
3,Professional growth within the medical sector,"The practical integration workshops were highly beneficial, they significantly contributed to a deeper comprehension of the theories and their implementation in a healthcare environment.",Incorporating a few advanced projects as optional tasks could benefit learners who wish to delve deeper into the subject matter. These projects wouldn't need to influence exam scores.
4,The highlights of the class included the practical demonstration clips that made the complex biological principles more understandable by connecting them to daily well-being and actions. This connection was incredibly beneficial as I navigated the course content.,"The aspect of the course that most facilitated my learning was the regular assessments provided at each segment, which helped confirm my grasp of the material presented. These checkpoints effectively guided me in the correct learning direction. It's evident that considerable effort was invested in designing these educational modules to enable students to gain a deep comprehension rather than just a superficial understanding of the subject matter.","Extend the duration of the concept videos for the more challenging topics, as they require a deeper dive to fully grasp the intricacies involved. Additionally, consider introducing an additional educator to the mix. The dynamic of having multiple voices in another subject area is quite engaging, and it would be beneficial to replicate that experience in this subject to prevent monotony from setting in with just one instructor."


We'll also load up some Coursera comments (source is from [this Kaggle notebook](https://www.kaggle.com/datasets/imuhammad/course-reviews-on-coursera), just using the first 200 as an example. The included example dataset is just the first 200 rows of the full 1.45 million rows. I didn't include the full set so as not to blimp up the size of this repo.

In [8]:
coursera_survey = pd.read_csv(data_path / 'coursera_survey_200rows.csv', nrows=200)
full_show(coursera_survey.head())

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just two complete watches so I'm happy about that. As usual there were some questions on the final exam that were NO WHERE in the course, which is annoying but far better than many microsoft tests I have taken. Never found the suplimental material that the course references... but who cares... i passed!",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and screen shots would sho on the side of the text that the instructor is going thru so that user does not have to go all the way to beginning of text to be able to view any slides instructor is showing.,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself was a little annoying. I had to wait 30 to 45 minutes after watching the videos to to take the quiz. Other than that the information was perfect and passed the test with no issues!,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do a double take but all in all not bad.,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was very detailed and easy to follow.,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


## Sentiment analysis

### Running

Let's run sentiment analysis on a single comment to see how it works. Notice that we record the model's reasoning as well to get a sense of how the results were arrived at.

In [9]:
survey_task = SentimentAnalysis(question="What could be improved about the course?")
task_input = CommentModel(comment=example_survey.iloc[0]['improve_course'])
llm_config = LLMConfig(logprobs=True, top_logprobs=3)

sample_sentiment = await apply_task_with_logprobs(task_input=task_input,
                                                  get_prompt=survey_task.prompt_messages,
                                                  result_class=survey_task.result_class,
                                                  llm_config=llm_config)

print(f'Student comment: "{task_input.comment}"')
pprint(json.loads(sample_sentiment.model_dump_json(include={'sentiment', 'reasoning'})))

Student comment: "Consider reducing the duration of certain videos. A few appeared to be slightly prolonged."
{'reasoning': 'The comment suggests a specific improvement (reducing the '
              'duration of certain videos) and mentions that some videos '
              'appeared to be slightly prolonged. The tone is constructive and '
              'not overly critical, indicating a neutral sentiment.',
 'sentiment': 'neutral'}


You can get a lot more detail from the result, but it's entirely optional - you can also just get the sentiment as above. Just for curiosity, here's some more of the detail. We can see that the top-ranked choice for the model was 'neutral' but next on its list was 'negative', hence we can do some finer-grained classification to call this comment 'neutral-negative', meaning that it's neutral but potentially negative-leaning.

In [10]:
print('logprobs for sentiment token:')
pprint(sample_sentiment.sentiment_logprobs)
print(f'fine-grained sentiment category: {sample_sentiment.fine_grained_sentiment_category}')

logprobs for sentiment token:
[{'linear_prob': 100.0, 'logprob': 0.0, 'token': 'neutral'},
 {'linear_prob': 0.0, 'logprob': -20.0, 'token': 'negative'},
 {'linear_prob': 0.0, 'logprob': -20.75, 'token': 'ne'}]
fine-grained sentiment category: neutral-negative


### Sorting/ranking

Next let's run this on a small set of comments and look at sorting/ranking. We'll take 50 comments from the survey question related to 'what were the best parts of the course?'

In [11]:
best_parts_question = "What were the best parts of the course?"
comments = example_survey['best_parts'].tolist()[:50]
sentiment_results = await classify_sentiment(comments=comments, question=best_parts_question)

processing 50 inputs in batches of 100
sleeping for 30 seconds between batches
starting 0 to 100
completed 0 to 100
elapsed time: 2.6933209896087646


We'll sort by the finer-grained sentiment. (For those interested, this is the difference in confidence between the top logprob sentiment and the next different sentiment/token, considering the top three only.) We'll only show the top 2 and bottom 2 comments output for brevity.

In [12]:
pairs = sort_by_confidence(comments, sentiment_results)
for comment, result in [*pairs[:2], *pairs[-2:]]:
    print(f'Student comment: "{comment}"')
    pprint(json.loads(result.model_dump_json(exclude={'logprobs'})))
    print('\n')

Student comment: "The flexibility to learn at a self-determined speed"
{'classification_confidence': {'difference': 'Infinity',
                               'next_token': None,
                               'top_token': 'positive'},
 'fine_grained_sentiment_category': 'positive',
 'reasoning': 'The comment highlights a positive aspect of the course, '
              "specifically the flexibility to learn at one's own pace. This "
              'indicates satisfaction and appreciation for this feature.',
 'sentiment': 'positive',
 'sentiment_logprobs': [{'linear_prob': 100.0,
                         'logprob': 0.0,
                         'token': 'positive'},
                        {'linear_prob': 0.0,
                         'logprob': -22.625,
                         'token': 'posit'},
                        {'linear_prob': 0.0,
                         'logprob': -24.375,
                         'token': 'Positive'}]}


Student comment: "Professional growth within the medic

### Displaying results

Create a display helper function for pretty output ranked by sentiment from most positive to most negative

In [13]:
color_map = {'positive': 'MediumSlateBlue', 
            'neutral-positive': 'LightSkyBlue',
            'neutral': 'LightGray',
            'neutral-negative': 'LightSalmon',
            'negative': 'Red'}
# this one is a diverging rdbu color map
# color_map = {'positive': '#0571b0', 
#             'neutral-positive': '#92c5de',
#             'neutral': '#f7f7f7',
#             'neutral-negative': '#f4a582',
#             'negative': '#ca0020'}

def colorized_html_output(pairs: tuple[str, SentimentAnalysisResult]) -> Any:
    html_output = ""
    for comment, result in pairs:
        color = color_map[result.fine_grained_sentiment_category]
        html_output += f'<span style="color:{color}; font-size:1.2em">"{comment}"</span><br>'

    return HTML(html_output)

def reference_colors():
    """This is kind of like a legend for the colorized_html_output"""
    html_output = ""
    for category, color in color_map.items():
        html_output += f'<span style="color:{color}; font-size:1.2em">{category}</span><br>'

    return HTML(html_output)

In [14]:
display(reference_colors())

Now let's take a look at the comments in a prettier format sorted from most positive (bluest in this scheme) to most negative (red in this scheme). Given that we are using the 'best parts' responses, one would expect that the vast majority would show up as positive sentiment.

In [15]:
display(colorized_html_output(pairs))

The color-coded version makes it pretty clear that most of the comments came across as positive, given that we were asking 'what were the best parts of the course?'

### Comparing to another survey question

Now let's take a look at the answers to "What could we improve about the course?". We might expect that there would be more comments that skew toward negative sentiment. Let's find out.

In [16]:
improve_course_question = 'What could be improved about the course?'
comments_improve = example_survey['improve_course'].tolist()[:50]
sentiment_results_improve = await classify_sentiment(comments=comments_improve, question=improve_course_question)


processing 50 inputs in batches of 100
sleeping for 30 seconds between batches
starting 0 to 100
completed 0 to 100
elapsed time: 3.2410290241241455


Rank the comments using the confidence difference scores within each top sentiment. Again, we'll only show the top 2 and bottom 2 comments for brevity.

In [18]:
pairs_improve = sort_by_confidence(comments_improve, sentiment_results_improve)
for comment, result in [*pairs_improve[:2], *pairs_improve[-2:]]:
    print(f'Student comment: "{comment}"')
    pprint(json.loads(result.model_dump_json(exclude={'logprobs'})))
    print('\n')

Student comment: "Its perfect this way"
{'classification_confidence': {'difference': 18.75,
                               'next_token': 'neutral',
                               'top_token': 'positive'},
 'fine_grained_sentiment_category': 'positive',
 'reasoning': 'The comment expresses satisfaction with the current state of '
              'the course, indicating no need for improvement. This suggests a '
              'positive sentiment.',
 'sentiment': 'positive',
 'sentiment_logprobs': [{'linear_prob': 100.0,
                         'logprob': 0.0,
                         'token': 'positive'},
                        {'linear_prob': 0.0,
                         'logprob': -18.75,
                         'token': 'neutral'},
                        {'linear_prob': 0.0,
                         'logprob': -19.0,
                         'token': 'negative'}]}


Student comment: "Everything is ideal as it stands."
{'classification_confidence': {'difference': 17.0,
             

In [19]:
display(reference_colors())

In [20]:
display(colorized_html_output(pairs_improve))

Sure enough - there are more comments that could be construed as negative, given that's what the "What could be improved about the course?" survey question is prompting for.

## Sentiment analysis for some Coursera comments

These are comments all from one course. There are a lot of duplicate rows in the dataset, for some reason, hence the drop_duplicates.

In [21]:
coursera_review_question = 'What did you think of the course?'
comments_coursera = coursera_survey.iloc[100:200].drop_duplicates()['reviews'].tolist()
sentiment_results_coursera = await classify_sentiment(comments=comments_coursera, question=coursera_review_question)

processing 54 inputs in batches of 100
sleeping for 30 seconds between batches
starting 0 to 100
completed 0 to 100
elapsed time: 3.676455020904541


Again, we'll sort by the finer-grained sentiment. We'll only show the top 2 and bottom 2 comments output for brevity.

In [22]:
pairs_coursera = sort_by_confidence(comments_coursera, sentiment_results_coursera)
for comment, result in [*pairs_coursera[:2], *pairs_coursera[-2:]]:
    print(f'Student comment: "{comment}"')
    print(f'Sentiment: {result.sentiment}')
    print(f'Fine-grained sentiment: {result.fine_grained_sentiment_category}')
    pprint(json.loads(result.model_dump_json(exclude={'logprobs', 'sentiment', 'fine_grained_sentiment_category'})))
    print('\n')

Student comment: "Good"
Sentiment: positive
Fine-grained sentiment: positive
{'classification_confidence': {'difference': 'Infinity',
                               'next_token': None,
                               'top_token': 'positive'},
 'reasoning': "The comment 'Good' is brief but indicates a positive experience "
              "with the course. The word 'Good' generally conveys satisfaction "
              'and approval.',
 'sentiment_logprobs': [{'linear_prob': 100.0,
                         'logprob': 0.0,
                         'token': 'positive'},
                        {'linear_prob': 0.0,
                         'logprob': -20.5,
                         'token': 'Positive'},
                        {'linear_prob': 0.0,
                         'logprob': -20.75,
                         'token': 'posit'}]}


Student comment: "pretty good"
Sentiment: positive
Fine-grained sentiment: positive
{'classification_confidence': {'difference': 17.375,
                      

In [23]:
display(reference_colors())

In [24]:
display(colorized_html_output(pairs_coursera))

^^ Pretty negative sentiment overall. On a practical level, we can see that the model has classified the comments well.

Let's try it for a different course.

In [25]:
coursera_review_question = 'What did you think of the course?'
comments_coursera2 = coursera_survey.iloc[:86].drop_duplicates()['reviews'].tolist()
sentiment_results_coursera2 = await classify_sentiment(comments=comments_coursera2, question=coursera_review_question)

processing 33 inputs in batches of 100
sleeping for 30 seconds between batches
starting 0 to 100
completed 0 to 100
elapsed time: 3.0723140239715576


In [26]:
display(reference_colors())

In [27]:
pairs_coursera2 = sort_by_confidence(comments_coursera2, sentiment_results_coursera2)
display(colorized_html_output(pairs_coursera2))

The comments' sentiment for this course seems much more positive.

We can also turn this into a dataframe for downloading or easier viewing. This is in the same order as the original dataframe, but you can use the sorted pairs and turn those into a dataframe if you want to keep the sort order shown above.

In [28]:
# here we omit the logprobs, as they are quite large
comments_df = pd.DataFrame({'comment': comments_coursera2})
values_df = pd.json_normalize([result.model_dump(exclude={'logprobs'}) for result in sentiment_results_coursera2])
results_df = pd.concat([comments_df, values_df], axis=1)

full_show(results_df.head())

,comment,reasoning,sentiment,sentiment_logprobs,fine_grained_sentiment_category,classification_confidence.top_token,classification_confidence.next_token,classification_confidence.difference
0,"Pretty dry, but I was able to pass with just two complete watches so I'm happy about that. As usual there were some questions on the final exam that were NO WHERE in the course, which is annoying but far better than many microsoft tests I have taken. Never found the suplimental material that the course references... but who cares... i passed!","The comment contains mixed sentiments. The student expresses some dissatisfaction with the course being 'pretty dry' and mentions annoyance about questions on the final exam that were not covered in the course. However, the student also expresses happiness about passing the course and notes that it was better than other tests they have taken. Overall, the sentiment leans towards neutral because the positive aspects (passing the course and it being better than other tests) balance out the negative aspects (dry content and irrelevant exam questions).",neutral,"[{'token': 'neutral', 'logprob': 0.0, 'linear_prob': 100.0}, {'token': 'ne', 'logprob': -23.875, 'linear_prob': 0.0}, {'token': 'negative', 'logprob': -24.0, 'linear_prob': 0.0}]",neutral,neutral,ne,23.875
1,would be a better experience if the video and screen shots would sho on the side of the text that the instructor is going thru so that user does not have to go all the way to beginning of text to be able to view any slides instructor is showing.,"The comment suggests an improvement to the course format, specifically regarding the placement of videos and screenshots. The student does not express any positive feelings about the current setup and focuses on a negative aspect that affects their experience. This indicates a negative sentiment.",negative,"[{'token': 'negative', 'logprob': 0.0, 'linear_prob': 100.0}, {'token': 'neutral', 'logprob': -17.5, 'linear_prob': 0.0}, {'token': 'neg', 'logprob': -20.375, 'linear_prob': 0.0}]",negative,negative,neutral,17.5
2,Information was perfect! The program itself was a little annoying. I had to wait 30 to 45 minutes after watching the videos to to take the quiz. Other than that the information was perfect and passed the test with no issues!,"The comment contains both positive and negative aspects. The student praises the information provided in the course, describing it as 'perfect' and mentions passing the test with no issues, which indicates a positive sentiment. However, the student also expresses frustration with the program's structure, specifically the wait time before taking the quiz, which is a negative sentiment. Overall, the positive aspects seem to outweigh the negative ones, leading to a generally positive sentiment.",positive,"[{'token': 'positive', 'logprob': -7.89631e-07, 'linear_prob': 100.0}, {'token': 'neutral', 'logprob': -14.250001, 'linear_prob': 0.0}, {'token': 'posit', 'logprob': -16.625, 'linear_prob': 0.0}]",positive,positive,neutral,14.25
3,A few grammatical mistakes on test made me do a double take but all in all not bad.,"The comment mentions some issues with grammatical mistakes on the test, which caused some confusion. However, the overall tone is not overly negative, as the student concludes with 'all in all not bad,' indicating a generally acceptable experience.",neutral,"[{'token': 'neutral', 'logprob': -3.1281633e-07, 'linear_prob': 100.0}, {'token': 'positive', 'logprob': -15.375, 'linear_prob': 0.0}, {'token': 'ne', 'logprob': -17.25, 'linear_prob': 0.0}]",neutral-positive,neutral,positive,15.375
4,Excellent course and the training provided was very detailed and easy to follow.,"The comment uses positive language such as 'Excellent course', 'very detailed', and 'easy to follow', indicating a high level of satisfaction with the course.",positive,"[{'token': 'positive', 'logprob': 0.0, 'linear_prob': 100.0}, {'token': 'posit', 'logprob': -20.375, 'linear_prob': 0.0},